In [946]:
appliances = {
        # 0: {'app': 'kettle', 'power': 2000},
        1: {'app': 'toaster', 'power': 1200},
        2: {'app': 'dish washer', 'power': 2000},
        # 0: {'app': 'kettle', 'power': 2300},
        # 1: {'app': 'toaster', 'power': 1500},
        # 2: {'app': 'dish washer', 'power': 2300},
        # 3: {'app': 'microwave', 'power':1500}
        # 4: {'app': 'hair dryer', 'power': 1000},
        # 5: {'app': 'audio amplifier', 'power': 20},
    }

#air dryer nao tem na casa 2?
#tirar hair dryer e audio amplifier!!!
#falta testar onde fica o dropout da ann!!

machine_learning_technique = ['logisticRegression', 'KNN', 'SVM', 'SGD', 'randomForest', 'perceptron']

homes = [1]#,2,5]

#esse dict soh serve pra dar load na rede
app_to_image_size = {
    'kettle': 48,
    'toaster': 64,
    'audio amplifier': 0,
    'dish washer': 166,
    'hair dryer': 27,
    'microwave': 42
}

In [947]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from matplotlib import rcParams
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import nilmtk
from nilmtk import DataSet, MeterGroup
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.linear_model import Perceptron, LogisticRegression, SGDClassifier, PassiveAggressiveClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import cross_val_score
import seaborn as sns

sns.set()
plt.style.use('ggplot')
rcParams['figure.figsize'] = (20, 8)
caminho_ukdale = 'ukdale.h5'

# Lendo Dataset do Ukdale
ukdale = DataSet(caminho_ukdale)

In [948]:
def create_dataset(ukdale, appliance, power, house):
    # Lendo eletrodomesticos da Casa House
    elec = ukdale.buildings[house].elec

    #Criando dataset do mains
    main_df = next(elec.mains().load())
    # print(main_df.head())
    main_df.columns = [' '.join(col).strip() for col in main_df.columns.values]
    main_df = main_df.rename(columns={'power apparent': 'pwr_app_mains', 'power active': 'pwr_act_mains', 'voltage': 'voltage_mains'})
    main_df = main_df.sort_index()
    
    appliance_df = next(elec[appliance].load())
    appliance_df.columns = ['pwr_act_' + appliance]
    appliance_df = appliance_df.sort_index()
    
    #-------------------Upsample do main_df--------------------------#
    appliance_df = appliance_df[appliance_df.index >= main_df.index[0]]

    main_df = main_df.resample('6S', offset='5s').mean().interpolate()
    main_df = main_df[main_df.index >= main_df.index[1]]
    main_df = main_df[main_df.index <= appliance_df.index[-1]]

    #------------------Resampling appliance_df-----------------------#
    appliance_df = appliance_df.resample('6S', offset='5s').mean().interpolate()

    #-------------------Merging dataframes---------------------------#
    merged_df = main_df.merge(appliance_df, left_index=True, right_index=True)
    merged_df['time'] = merged_df.index
    merged_df['time'] = merged_df['time'].apply(lambda x: x.tz_localize(None))
    merged_df.index = merged_df['time']
    merged_df['dia_semana'] = merged_df.time.dt.dayofweek
    merged_df['mes'] = merged_df.time.dt.month
    merged_df['hora'] = merged_df.time.dt.hour
    merged_df['timestamp'] = merged_df['time'].apply(lambda x: x.timestamp())
    merged_df = merged_df.drop(columns=['time'])

    #---------------Criando coluna classificatoria-------------------#
    merged_df['ligado'] = merged_df['pwr_act_' + appliance] > power
    
    return merged_df

In [949]:
def separate_training_from_test(df):
    #Fazendo a divisao entre treino e teste de forma que 1 milhao de pontos sejam utilizados para treinar
    train_test_divide = df.index[int(0.8*len(df))]

    print("inicio treino: " + str(df.index[0]))
    print("final treino: " + str(train_test_divide))
    print("final teste: " + str(df.index[-1]))
    
    train = df[df.index <= train_test_divide]
    test = df[df.index > train_test_divide]
    
    return train,test

In [950]:
def create_sequential_dataset(df):
    df = df[['voltage_mains', 'pwr_app_mains', 'pwr_act_mains', 'dia_semana', 'hora', 'ligado']]
    df_scaled = pd.DataFrame(np.append(
        preprocessing.scale(df[['voltage_mains', 'pwr_app_mains', 'pwr_act_mains', 'dia_semana', 'hora']]), df[['ligado']].values, axis=1),
                                index=df.index, columns=df.columns)

    return df_scaled

In [951]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import loguniform
from sklearn.model_selection import RepeatedStratifiedKFold

def choose_net(ml_tec, appliance):
    if ml_tec == 'logisticRegression':
        if appliance == 'kettle':
            return LogisticRegression(class_weight={0: 0.2, 1: 0.8}, multi_class='multinomial', penalty='none', solver='lbfgs', n_jobs=-1) #kettle
        if appliance == 'toaster':
            return LogisticRegression(class_weight={0: 0.15, 1: 0.85}, multi_class='multinomial', penalty='none', solver='newton-cg', n_jobs=-1) #toaster
        if appliance == 'dish washer':
            return LogisticRegression(class_weight={0: 0.15, 1: 0.85}, multi_class='multinomial', penalty='none', solver='newton-cg', n_jobs=-1) #dishWasher


    if ml_tec == 'KNN':  # demorado
        if appliance == 'kettle':
            return KNeighborsClassifier(weights='distance', p=3, n_neighbors=9, metric='minkowski', leaf_size=40, algorithm='kd_tree', n_jobs=-1) #kettle
        if appliance == 'toaster':
            return KNeighborsClassifier(weights='distance', p=4, n_neighbors=11, metric='manhattan', leaf_size=50, algorithm='ball_tree', n_jobs=-1) #toaster
        if appliance == 'dish washer':
            return KNeighborsClassifier(weights='uniform', p=5, n_neighbors=11, metric='minkowski', leaf_size=50, algorithm='kd_tree', n_jobs=-1) #dishWasher 0.725420462724261


    if ml_tec == 'SVM':
        if appliance == 'kettle':
            return LinearSVC(penalty='l2', multi_class='crammer_singer', loss='squared_hinge', dual=True, class_weight={0: 0.25, 1: 0.75}) #kettle
        if appliance == 'toaster':
            return LinearSVC(penalty='l2', multi_class='crammer_singer', loss='squared_hinge', dual=True, class_weight={0: 0.2, 1: 0.8}) #toaster
        if appliance == 'dish washer':
            return LinearSVC(penalty='l1', multi_class='crammer_singer', loss='squared_hinge', dual=False, class_weight={0: 0.2, 1: 0.8}) #dishWasher

    if ml_tec == 'SGD':
        if appliance == 'kettle':
            return SGDClassifier(penalty=None, loss='hinge', l1_ratio=0.1, class_weight= {0: 0.15, 1: 0.85}, n_jobs=-1)
        if appliance == 'toaster':
            return SGDClassifier(penalty=None, loss='modified_huber', class_weight= {0: 0.15, 1: 0.85}, n_jobs=-1) #toaster
        if appliance == 'dish washer':
            return SGDClassifier(penalty=None, loss='hinge', class_weight= {0: 0.15, 1: 0.85}, n_jobs=-1) #dishWasher

    if ml_tec == 'randomForest': # demorado
        if appliance == 'kettle':
            return RandomForestClassifier(criterion='gini', max_features='sqrt', n_estimators=300, class_weight={0: 0.2, 1: 0.8}, n_jobs=-1) #kettle
        if appliance == 'toaster':
            return RandomForestClassifier(criterion='entropy', max_features='sqrt', n_estimators=300, class_weight={0: 0.3, 1: 0.7}, n_jobs=-1) #toaster
        if appliance == 'dish washer':
            return RandomForestClassifier(criterion='entropy', max_features='sqrt', n_estimators=300, class_weight={0: 0.2, 1: 0.8}, n_jobs=-1) #dishWasher

    if ml_tec == 'perceptron':
        if appliance == 'kettle':
            return Perceptron(n_jobs=-1, penalty='elasticnet', l1_ratio=0.15, class_weight={0: 0.3, 1: 0.7}) #kettle
        if appliance == 'toaster':
            return Perceptron(n_jobs=-1, penalty='l1', l1_ratio=0.1, class_weight={0: 0.25, 1: 0.75}) #toaster
        if appliance == 'dish washer':
            return Perceptron(n_jobs=-1, penalty=None, l1_ratio=0.1, class_weight={0: 0.15, 1: 0.85}) #dishWasher

# def parameter_grid_svm():
#     random_grid = {
#                'penalty' : ['l1','l2'],
#                'loss' : ['hinge', 'squared_hinge'],
#                'dual': [False],# 'False'],
#                'multi_class' : ['crammer_singer'],#['ovr','crammer_singer'],
#                'class_weight': [{0:.2, 1:.8}, {0:.25, 1:.75}],#[{0:.15, 1:.85}, {0:.2, 1:.8}, {0:.25, 1:.75}, {0:.3, 1:.7}]# {0:.35, 1:.65}] 
#                }
#     return random_grid

# def parameter_grid_logistic_regression():
#     random_grid = dict()
#     random_grid['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
#     random_grid['penalty'] = ['none']#, 'l1', 'l2', 'elasticnet']
#     # random_grid['C'] = [0.005, 0.05, 0.5, 1, 5]
#     random_grid['multi_class'] = ['multinomial']#, 'ovr']
#     random_grid['class_weight'] = [{0:.15, 1:.85}, {0:.2, 1:.8}, {0:.25, 1:.75}] 
#     return random_grid

# def parameter_grid_knn():
#     random_grid = {'n_neighbors' : [11], #[9,11,15],#[5,7,9,11,15],
#                'weights' : ['uniform', 'distance'],
#                'algorithm': ['ball_tree', 'kd_tree'],#['auto', 'ball_tree', 'kd_tree'],
#                'metric' : ['minkowski','euclidean','manhattan'],
#                'p': [3, 4, 5],#[2, 3, 4, 5],
#                'leaf_size': [40, 50],# 60],#[30, 40, 50, 60]
#                }
#     return random_grid

# def parameter_grid():
#     random_grid = {
#                'penalty' : [None],#['l1','l2', 'elasticnet', None],
#                'loss' : ['hinge'],#['hinge', 'log_loss', 'log', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
#             #    'alpha': [0.0001, 0.001, 0.01, 0.1],
#             #    'l1_ratio': [0.05, 0.1, 0.15, 0.2],
#             #    'shuffle': [True],# False],
#             #    'learning_rate' : ['constant', 'optimal', 'invscaling', 'adaptive'],
#             #    'early_stopping': [True, False],
#                'class_weight': [{0:.15, 1:.85}, {0:.2, 1:.8}, {0:.25, 1:.75}, {0:.3, 1:.7}, {0:.35, 1:.65}] 
#                }
#     return random_grid

# def parameter_grid_rf():
#     random_grid = {
#                'criterion' : ['gini', 'entropy', 'log_loss'],
#                'max_features' : ['sqrt', None],# 'log2', None],
#                'class_weight': [{0:.2, 1:.8}, {0:.3, 1:.7}],# [{0:.15, 1:.85}, {0:.2, 1:.8}, {0:.25, 1:.75}, {0:.3, 1:.7}],# {0:.35, 1:.65}],
#                'n_estimators': [300],#[50,100,200,300]
#                }
#     return random_grid

# def parameter_grid_percebetron():
#     random_grid = {
#                'penalty' : ['l1','l2', 'elasticnet', None],
#             #    'alpha': [0.0001, 0.001, 0.01, 0.1],
#                'l1_ratio': [0.1, 0.15],#[0.05, 0.1, 0.15, 0.2],
#             #    'shuffle': [True, False],
#             #    'early_stopping': [True, False],
#                'class_weight': [{0:.15, 1:.85}, {0:.2, 1:.8}, {0:.25, 1:.75}, {0:.3, 1:.7}, {0:.35, 1:.65}] 
#                }
#     return random_grid

def randomize_search(estimator, random_grid, dataset):
    # cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    model_random = RandomizedSearchCV(estimator = estimator,param_distributions = random_grid,
               n_iter = 10, cv = 5, scoring='f1', verbose=2, random_state=8, n_jobs = -1)
    model_grid = GridSearchCV(estimator = estimator,param_grid = random_grid, cv = 5, scoring='f1', verbose=3, n_jobs = -1)
    return model_grid.fit(dataset[['voltage_mains', 'pwr_app_mains', 'pwr_act_mains', 'dia_semana', 'hora']], dataset[['ligado']])




In [952]:
def train_net(net, dataset, appliance):
    comeco = datetime.datetime.now()
    print('###########################')
    print('Comeco do treino para ', appliance, ' utilizando a rede ', ml_tec, ' inicio as ', comeco)

    net.fit(dataset[['voltage_mains', 'pwr_app_mains', 'pwr_act_mains', 'dia_semana', 'hora']], dataset[['ligado']])

    duracao = datetime.datetime.now() - comeco
    print('Finished Training ', ml_tec, ' para ', appliance, ' com duracao de ', duracao)
    print('terminou as ', datetime.datetime.now())
    return net

In [953]:

def print_start_test(appliance, ml_tec, house, is_test=False):
    comeco = datetime.datetime.now()
    print("-----------------------------")
    if is_test == True:
        print('comeco do teste para ', appliance, 'utilizando ', ml_tec, 'foi as', comeco)
    else:
        print('comeco da validacao para casa', str(house), 'do ', appliance, 'utilizando ', ml_tec, 'foi as', comeco)


def test_net(net, dataset):
    comeco_teste = datetime.datetime.now()

    array_results = net.predict(dataset[['voltage_mains', 'pwr_app_mains', 'pwr_act_mains', 'dia_semana', 'hora']])

    duracao = datetime.datetime.now() - comeco_teste
    print('duracao do teste:', duracao)
    print("-----------------------------")
    return array_results

def out_into_df(df, out):
    df['saidas'] = out
    return df

def save_df_into_csv(df, ml_tec, appliance, house, is_test=False):
    if is_test == True:
        nome_arquivo = 'saida_teste_' #TODO

    else:
        nome_arquivo = 'saida_casa_' + str(house) + '_'

    nome_appliance = appliance.replace(' ', '_')
    nome_arquivo = 'results/csv/'+nome_arquivo + ml_tec + '_' + nome_appliance + '_v2.csv'

    df.to_csv(nome_arquivo, sep=',', header=True, index=True)
    return True

def open_csv_as_df(ml_tec, appliance, house, is_test=False):
    if is_test == True:
        nome_arquivo = 'saida_teste_'
    else:
        nome_arquivo = 'saida_casa_' + str(house) + '_'
    nome_appliance = appliance.replace(' ', '_')
    nome_arquivo = nome_arquivo + ml_tec + '_' + nome_appliance + '.csv'
    df = pd.read_csv(nome_arquivo, sep=',')
    return df

In [954]:
def save_results_txt(text, appliance, ml_tec, house = 1, test = False):  
    file_name = 'results_'
    nome_appliance = appliance.replace(' ', '_')
    if test == True:
        file_name = file_name + '_teste_'
    file_name = file_name + 'casa_' + str(house) + '_'
    file_name = file_name + ml_tec + '_' + nome_appliance + '.txt'
    f = open(file_name, 'w+')  # open file in write mode
    f.write(text)
    f.close()
    print('resultados salvos com sucesso em:', file_name)

In [955]:
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score  

def calculate_metrics(dataset):
    dataset["ligado"] = dataset["ligado"].astype(int)
    y = dataset.loc[:,('ligado')].values
    y_pred = dataset.loc[:,('saidas')].values


    f1 = f1_score(y, y_pred)
    accuracy = accuracy_score(y, y_pred)
    recall = recall_score(y, y_pred)
    precision = precision_score(y, y_pred)

    
    print(f1, accuracy, recall, precision)

In [956]:
comeco_total = datetime.datetime.now()
print('comecou as ', comeco_total)

results_df = pd.DataFrame(columns=['app', 'tec', 'house', 'recall', 'accuracy', 'precision', 'f1', 'TP', 'TN', 'FP', 'FN', 'P', 'N'])
for i in appliances:
    comeco_aparelho = datetime.datetime.now()
    
    ukdale = DataSet(caminho_ukdale)

    #Escolha se deseja escolher uma janela de tempo e o período de inicio e fim:    
    set_timewindow = True
    startDate = '2013-03-01'
    endDate = '2013-09-01'
    
    #Escolha do eletrodomestico:
    appliance = appliances[i]['app']
    power = appliances[i]['power']
    
    if set_timewindow:
        ukdale.set_window(start=startDate, end=endDate)
    
    print('Comeco do aparelho ', appliance , ' foi as ', comeco_aparelho)
    #create Dataset
    df = create_dataset(ukdale, appliance, power, 1)
    
    #Desagregacao
    train,test = separate_training_from_test(df)
    
    trainloader = create_sequential_dataset(train)
    # trainloader = create_sequential_dataset(df)

    ####running all for all appliances#####
    for ml_tec in machine_learning_technique:
        
        #training
        net = choose_net(ml_tec, appliance)
        net = train_net(net, trainloader, appliance)

        #validating
        # random_grid = parameter_grid()
        # randomized_net = randomize_search(net, random_grid, trainloader)
        # print(randomized_net.best_params_)
        # print(randomized_net.best_score_)

        # testing
        print_start_test(appliance, ml_tec, 1, is_test=True)
        testloader = create_sequential_dataset(test)
        out = test_net(net, testloader)
        test = out_into_df(test, out)
        save_df_into_csv(test, ml_tec, appliance, 1, is_test=True)

        # # resultados em casas nao vistas
        for house in homes:
            if house == 1:
                set_timewindow = True
                startDate = '2013-09-01'
                endDate = '2015-05-01'
            else:
                set_timewindow = False
            ukdale = DataSet(caminho_ukdale)
            if set_timewindow:
                ukdale.set_window(start=startDate, end=endDate)
            df = create_dataset(ukdale, appliance, power, house)

            print_start_test(appliance, ml_tec, house)
            dfloader = create_sequential_dataset(df)
            out = test_net(net, dfloader)
            df = out_into_df(df, out)
            save_df_into_csv(df, ml_tec, appliance, house)
        
print('fim do aparelho ', appliance, 'as ', datetime.datetime.now())
print('duracao de ', datetime.datetime.now() - comeco_aparelho)

comecou as  2023-04-12 19:05:01.175667
Comeco do aparelho  toaster  foi as  2023-04-12 19:05:01.177837


KeyboardInterrupt: 

{'class_weight': {0: 0.15, 1: 0.85}, 'loss': 'hinge', 'penalty': None}
0.6047103715335518


In [ ]:

0.6094313157178869

0.6094313157178869